# Controller Design for a Magnetic Levitation Kit using OpenModelica’s Integration with the Julia Language

Jupyter notebook accompanying the corresponding conference talk on the 13th Int. Modelica Conference:

Bernhard Thiele, Bernt Lie, Martin Sjölund, Adrian Pop, and Peter Fritzson. Controller Design for a Magnetic Levitation Kit using OpenModelica’s Integration with the Julia Language. In Anton Haumer, editor, 13th Int. Modelica Conference, Regensburg, Germany, March 2019. [doi:10.3384/ecp19157303](https://www.doi.org/10.3384/ecp19157303).

As of 2019-03-05 the descriptive text is not complete (see the full paper for missing details), but the relevant code is included and can be executed.

Feedback for improving the notebook or other aspects of the design is welcome...

## Motivation: Is it possible to combine the strengths of Modelica and Julia?!

- Modelica
  - Well established for modeling complex technical systems using an equation-based declarative modeling style
  - Imperative part for writing algorithms can (in principle) be used for numerical computing tasks or scripting tasks
  - However, support in tools as scripting language has so far remained limited and rather tool specific
  - No rich ecosystem for typical numerical computing tasks like data analysis, advanced data visualization, control design
- Julia
  - Rather young language (Julia 1.0 was released in August 2018) designed to address the needs of numerical analysis and computational science
  - Convenient and concise syntax for working with arrays and matrices (similar to Matlab or Modelica)
  - Has attracted a growing user base in the scientific computing community
  - Rich ecosystem for numerical computing tasks, including decent support for the control community

  Approach taken at LiU: Develop a bridge from Julia to OpenModelica (OMJulia)  
  **Motivation: Evaluate approach using an interesting and fun control design use case**

## Use Case: Digital Control for a Magnetic Levitation Kit

- Magnetic levitation is a popular application for teaching control theory
- Underlying physics (unstable plant dynamics) convincingly demonstrate the importance of feedback control
- Application based on a commercially available electromagnetic levitation kit from Zeltom (http://zeltom.com/product/magneticlevitation)

<img src="images/MagLevKit.png" alt="Zeltom's Electromagnetic Levitation Kit" title="Zeltom's Electromagnetic Levitation Kit"/>

**Goal: Replace Zeltom’s controller by our own design**

## Plant Model (From Zeltom‘s Technical Documentation)

Approximated force from the electromagnet on the levitating magnet:

$f = k\frac{i}{d^4},$

approximated voltage across the Hall effect sensor:

$e = \alpha + \beta\frac{1}{d^2} + \gamma i,$

Newton’s second law:

$m\frac{\mathrm{d}^2 d}{\mathrm{d}\,t^2} = mg - f,$

Kirchhoff's voltage law:

$v = Ri + L\frac{\mathrm{d} i}{\mathrm{d}\,t},$


where $k$ is a geometry dependent constant, $\alpha,\beta,\gamma$ are constants that depend on the Hall sensor and geometry, $g$ is the standard gravity constant, $R$ is the resistance and $L$ the inductance of the electromagnet.

<img src="images/MagLevSchematics.png" alt="Schematic of the magnetic levitation system" title="Schematic of the magnetic levitation system"/>


## Corresponding Nonlinear Modelica Model

```modelica
model MagLevNL
  parameter Real R=2.41, L=15.03e-3, m=3.02e-3, k=17.31e-9, alpha=2.44, beta=1.12e-4, gamma=0.26;
  input Real v;
  output Real e;
  parameter Real i0, d0, d_der0;
  Real i(start=i0,fixed=true), d(start=d0,fixed=true), d_der(start=d_der0,fixed=true), f;
  constant Real g=9.81;
equation
  f = k*i/d^4;
  e = alpha + beta*1/d^2 + gamma*i;
  der(d) = d_der;
  m*der(d_der) = m*g - f;
  v = R*i + L*der(i);
end MagLevNL;
```

## OMJulia, an API for Interacting with Modelica Models from the Julia Language

- OMJulia is a Julia package which  provides a Julia API to OpenModelica
- Details: Bernt Lie, Arunkumar Palanisamy, Alachew Mengist, Lena Buffoni, Martin Sjölund, Adeel Asghar, Adrian Pop, and Peter Fritzson. OMJulia: An OpenModelica API for Julia-Modelica Interaction. In Anton Haumer, editor, _13th Int. Modelica Conference_, Regensburg, Germany, March 2019. [doi:10.3384/ecp19157699](http://dx.doi.org/doi:10.3384/ecp19157699).

Installs via the standard (Git-based) Julia package manager:

```julia
Pkg.clone("https://github.com/OpenModelica/OMJulia.jl")
```

**Low-level API**: Simply send OpenModelica script string commands

In [ ]:
using OMJulia

In [ ]:
omc = OMJulia.OMCSession()
sendExpression(omc, "getVersion()")

**High-Level API**: Create ModelicaSystems object

In [ ]:
mlNL = OMJulia.OMCSession()
ModelicaSystem(mlNL, "MagLevNL.mo",  "MagLevNL")
getParameters(mlNL)

Julia packages used within this notebook (need to be installed by the package manager):

- OMJulia
- ControlSystems
- Interact
- Plots
- PyPlot
- CSV
- LaTeXStrings

## Used Control Design Approach

1. Linearize plant around an equilibrium position at $d=0.02$ m
2. Design a stabilizing continuous-time controller using the linear model
3. Test and tune the synthesized controller by plugging it into the nonlinear Modelica model
4. Emulate the continuous-time controller by a discrete-time approximation
5. Test and tune the controller in a more detailed closed-loop sampled-data model
6. Generated real-time code for the Arduino Uno
7. Control the maglev system using an Arduino Uno and supporting electronics

## 1. Linearization ─ Find Equilibrium Position

Would like something similar to

```julia
mlNL = OMJulia.OMCSession()
ModelicaSystem(mlNL, "MagLevNL.mo", "MagLevNL")
state_e, u_e, y_e = findEquilibrium(mlNL, ["d=0.02", "d_der=0"])
```

But unfortunately no function `findEquilibrium` exists (yet).

Workaround: Use Modelica's steady-state initialization facilities for finding the desired equilibrium position.

Create an augmented version of the `MagLevNL` model with steady-state initialization:

```modelica
model MagLevNL_SteadyState
  parameter Real R=2.41, L=15.03e-3, m=3.02e-3, k=17.31e-9, alpha=2.44, beta=1.12e-4, gamma=0.26;
  parameter Real d0 = 0.02 "Prescribed equilibrium position";
  parameter Real v(start=0.5, fixed=false) "Unknown equilibrium voltage accross the electromagnet";
  output Real e;
  Real i, d, d_der, f;
  constant Real g=9.81;
equation
  f = k*i/d^4;
  e = alpha + beta*1/d^2 + gamma*i;
  der(d) = d_der;
  m*der(d_der) = m*g - f;
  v = R*i + L*der(i);
initial equation
  d = d0;
  der(d) = 0;
  der(d_der) = 0;
  der(i) = 0;
end MagLevNL_SteadyState;
```

Now following Julia code can be used for retrieving the linearized model:

In [ ]:
using OMJulia, Printf
mlNLe = OMJulia.OMCSession()
ModelicaSystem(mlNLe, "MagLevNL_SteadyState.mo", "MagLevNL_SteadyState")
setParameters(mlNLe, ["d0=0.02"])
simulate(mlNLe)
sol = getSolutions(mlNLe, ["v","i","d","d_der","e"])
v_e = sol[1][1] # input v at equilibrium
i_e = sol[2][1] # state i at equilibrium
d_e = sol[3][1] # state d, must be equal d0
d_der_e = sol[4][1] # state d_der, must be 0
e_e = sol[5][1] # output e at equilibrium
@printf("v_e=%0.2f, i_e=%0.2f, d_e=%0.2f, d_der_e=%0.2f, e_e=%0.2f\n", v_e, i_e, d_e, d_der_e, e_e)

mlNL = OMJulia.OMCSession()
ModelicaSystem(mlNL, "MagLevNL.mo","MagLevNL")
setInputs(mlNL, ["v=$v_e"])
setParameters(mlNL, ["i0=$i_e", "d0=$d_e", "d_der0=$d_der_e"])
A,B,C,D = linearize(mlNL)
println("A=", A, "\nB=", B, "\nC=", C, "\nD=", D)
#println("A=$A\nB=$B\nC=$C\nD=$D")


`linearize()` performs a simulation and retrieves a tuple of 2D arrays (matrices) encoding the linearized model ($\dot{x}=Ax+Bu, y=Cx+Du$) at the end of the simulation (`stopTime`).

A couple of options can be set, e.g., to set the `stopTime` at which the linearization is carried out.

## 2. Control Design

### Create linear time-invariant model of plant

Create LTI model from the previously retrieved matrices using the ControlSystems.jl package:

In [ ]:
using ControlSystems
mlLin = ss(A,B,C,D)

The magnetic levitation system is open-loop unstable:

In [ ]:
pole(mlLin)

The state space representation `mlLin` can be converted to a transfer function representation:

In [ ]:
G = tf(mlLin)

### Some Definitions

It is known that a *PD controller* is capable of stabilizing the magnetic levitation system at hand:

$C_{PD}(s) = K_p(T_d s+1),$

where $K_p$ is the proportional gain and $T_d$ the derivative time parameter.

Using the plant's transfer function $G(s)$, the open-loop transfer function is given by:

$P_{PDol} = C_{PD}(s) G(s)$.

As a measure for the *robustness* of a design the *sensitivity function* $S(s)$ is used (it describes the transfer function from an external disturbance to the process output). It is given by

$S(s) = \frac{1}{1+P_{PDol}(s)}$

**Approach: Use the sensitivity function for designing a reasonable robust controller.**

### Interactive Plots for Parameter Tuning

Combining the Interact.jl and ControlSystems.jl package enables interactive plots in which the controller’s parameters can be tuned experimentally

In [ ]:
using Interact
s = tf("s")
ui = @manipulate for Kp = 3:.5:20, Td = 0.01:.01:0.1
  PD = Kp*(Td*s + 1)
  mlLinPDol = series(PD,G)
  mlLinPDSensitivity = minreal(1/(1+mlLinPDol))
  bodeplot(mlLinPDSensitivity,plotphase=false,yscale=:identity,yticks = 0:0.2:4,title="Sensitivity")
end

## 3. Test PD Controller in the Nonlinear Closed-Loop Model

- The PD controller can be transcribed into Modelica code and can be added to the "MagLev" model
- Let the resulting model be named "MagLevNLPD"

In [ ]:
using OMJulia, Plots, Interact

mlNLPD = OMJulia.OMCSession()
ModelicaSystem(mlNLPD, "MagLevNLPD.mo","MagLevNLPD")
setSimulationOptions(mlNLPD, ["stopTime=0.5", "tolerance=1e-8"])
gr(size=(700,300))
@manipulate for Kp = 7:.5:23, Td = 0.01:.01:0.1, d0 = 0.015:0.0002:0.025
  setParameters(mlNLPD, ["Kp=$Kp", "Td=$Td", "d0=$d0"])
  simulate(mlNLPD)
  sol = getSolutions(mlNLPD, ["time", "d", "v"])
  time, d, v = sol[1], sol[2], sol[3]
  p1 = plot(time, d, label="", xlabel="time [s]", ylabel="d [m]")
  p2 = plot(time, v, label="", xlabel="time [s]", ylabel="v [V]")
  plot(p1,p2,layout=(1,2))
end

**Combining OMJulia with the Interact package allows to quickly create small GUIs for interactive experimentation with a Modelica model**

## 4. Digital Control

### Discrete-Time Approximation

For a practical implementation the derivative "$D$" part is first approximated by a "$DT_1$":

$sT_{d}\approx\frac{sT_{d}}{1+sT_{d}/N_{d}}$

where $N_d$ limits the gain at high frequencies (typically:
$3\leq N_{d}\leq 20$). Therefore, the structure of the controller becomes

$C_{PDT_{1}}(s) = K_P\left( \frac{T_{d}s}{\frac{T_{d}}{N_{d}}s + 1} + 1\right).$

Using backward differences the transfer function can be transformed into a pulse-transfer function by substituting $s$ by $s'$ using the formula

$s' = \frac{z - 1}{zh},$

where $h$ is the sampling period and $z$ is the Z-transform variable, resulting in the pulse-transfer function

$C_{PDT_{1}}(z) = K_p \left(\frac{T_d N_d(z - 1)}{(T_d + N_d h)z - T_d} + 1\right).$

The pulse-transfer function can be readily transformed into a recurrance relation which directly translates into Modelica code.

### Discretized controller using Modelica’s clocked synchronous language elements

```modelica
block Controller
  parameter Real Kp=15, Td=0.05, Nd=5, h=0.0005, v_e=0.66, e_e=2.79;
  input Real du_set "Setpoint delta voltage (=0 for d=>0.02)";
  input Real e "Measured voltage across the Hall effect sensor";
  output Real v "Output voltage to the electromagnet";
protected
  Real Dpart(start=0), de_e, du(start=0), dy, ad, bd;
equation
  // Measured delta voltage at OP
  de_e = e - e_e;
  // input to PD(T1) control law
  du = du_set - de_e;

  // Control law
  ad = Td/((Td + Nd*h));
  bd = Td*Nd/(Td + Nd*h);
  Dpart =  ad * previous(Dpart) + bd * (du - previous(du));
  dy = Kp*(du + Dpart);

  // Output voltage to electromagnet
  v = dy + v_e;
end Controller;
```

## 5. Sampled-Data Model

### Target Hardware Arduino Uno Board

- Microcontroller
  - ATmega328P (16 MHz), 2 KB SRAM, 32 KB flash memory
- Analog input pins
  - 6 (10-bit ADC, voltage range 0 V - 5 V)
- PWM digital I/O pins
  - 6 (PWM frequency configurable, PWM duty cycle can be set with a resolution of 8-bit)
- Supporting electroncis on breadboard
  - Voltage to the electromagnet is set by a PWM output driving a MOSFET which is connected to a DC voltage regulator fed from an external power supply

<img src="images/Arduino_Uno_-_R3.jpg" alt="Arduino Uno Board" title="Arduino Uno Board"/>

### Use of Modelica_Synchronous Library

Sample and hold blocks modeling following effects:

- Computational delay between input and output (one sample period).
- Actuating variable $v$ is limited between $0\,\mathrm{V}\leq v \leq 1.3\,\mathrm{V}$ using 8-bit quantization.
- ADC 10-bit resolution in $[0\,\mathrm{V}, 5\,\mathrm{V}]$ for measurement variable $e$.

<img src="images/MagLev_Diagram.png" alt="Closed-loop magnetic levitation system with clocked controller model" title="Closed-loop magnetic levitation system with clocked controller model"/>

Martin Otter, Bernhard Thiele, and Hilding Elmqvist. A Library for Synchronous Control Systems in Modelica. In Martin Otter and Dirk Zimmer, editors, _9th Int. Modelica Conference_, Munich, Germany, September 2012. [doi:10.3384/ecp1207627](http://dx.doi.org/10.3384/ecp1207627).

### Simulation results

Example of using the "low-level" API for creating Figure 6 in the paper (Simulation results of the sampled-data model for different ADC quantization settings and an initial distance d0 = 0.019 m):

1. Create simulation result files
2. Read result files and create plots

In [1]:
using OMJulia
AVRcl = OMJulia.OMCSession()

res = sendExpression(AVRcl, "loadModel(Modelica)")
if (!res) print(sendExpression(AVRcl, "getErrorString()")) end

res = sendExpression(AVRcl, "loadModel(Modelica_Synchronous)")
if (res)
    res = sendExpression(AVRcl, "getSourceFile(Modelica_Synchronous)")
    print("LoadModel: " * res)
else
    print(sendExpression(AVRcl, "getErrorString()"))
end

res = sendExpression(AVRcl, "loadFile(\"ML.mo\")")
if (res)
    res = sendExpression(AVRcl, "getSourceFile(ML)")
    print("LoadFile: " * res)
else
    print(sendExpression(AVRcl, "getErrorString()"))
end

res = sendExpression(AVRcl, "setLanguageStandard(\"latest\")")
if (!res) print(sendExpression(AVRcl, "getErrorString()")) end

mkpath("simdir")
sendExpression(AVRcl, "cd(\"simdir\")")
res = sendExpression(AVRcl, "buildModel(ML.MagLev_AVRcl, stopTime=5.0, fileNamePrefix=\"AVRcl\", outputFormat=\"csv\")")
print(sendExpression(AVRcl, "getErrorString()"))

cd("simdir")
run(`./AVRcl -r=../res_Default.csv`)
run(`./AVRcl -override=sample1.bits=16 -r=../res_e_0_5_16.csv`)
run(`./AVRcl -override=sample1.yMin=2.5,sample1.yMax=3.5 -r=../res_e_2.5_3.5_10.csv`)
cd("..")

LoadModel: /usr/lib/omlibrary/Modelica_Synchronous 0.92.1/package.moLoadFile: /home/bernhard/bt/dev/MagLevApp/ML.moNotification: Skipped loading package Modelica_DeviceDrivers (1.6.0,default) using MODELICAPATH /usr/bin/../lib/omlibrary:/home/bernhard/.openmodelica/libraries/ (uses-annotation may be wrong).
LOG_SUCCESS       | info    | The initialization finished successfully without homotopy method.
LOG_SUCCESS       | info    | The simulation finished successfully.
LOG_SUCCESS       | info    | The initialization finished successfully without homotopy method.
LOG_SUCCESS       | info    | The simulation finished successfully.
LOG_SUCCESS       | info    | The initialization finished successfully without homotopy method.
LOG_SUCCESS       | info    | The simulation finished successfully.


**Caution (2019-03-10):** Julia v1.1.0 might print a lot of deprecation warnings when executing the code below. The actual plot is then "hidden" in the warnings.

In [ ]:
using Plots, CSV, LaTeXStrings
pyplot()

d1 = CSV.read("res_Default.csv")
d2 = CSV.read("res_e_2.5_3.5_10.csv")
d3 = CSV.read("res_e_0_5_16.csv")

lp1 = latexstring("ADC 10-bit resolution in \$[0\\,\\mathrm{V},5\\,\\mathrm{V}]\$")
lp2 = latexstring("ADC 10-bit resolution in \$[2.5\\,\\mathrm{V},3.5\\,\\mathrm{V}]\$")
lp3 = latexstring("ADC 16-bit resolution in \$[0\\,\\mathrm{V},5\\,\\mathrm{V}]\$")
pa1 = plot(d1[:time], d1[Symbol("magLevNL.d")],  line=(:dot), label=lp1, legend=:topright, xlabel="time [s]", ylabel="magLevNL.d [m]")
pa2 = plot!(d2[:time], d2[Symbol("magLevNL.d")], line=(:dashdot), label=lp2)
pa3 = plot!(d3[:time], d3[Symbol("magLevNL.d")], line=(:solid), label=lp3)

pb1 = plot(d1[:time], d1[Symbol("sample1.y")], xlims = (4,4.03), ylims = (2.78,2.8), line=(:dot, :steppost), label="", xlabel="time [s]", ylabel="sample1.y [V]")
pb2 = plot!(d2[:time], d2[Symbol("sample1.y")], xlims = (4,4.03), ylims = (2.78,2.8), line=(:dashdot, :steppost), label="")
pb3 = plot!(d3[:time], d3[Symbol("sample1.y")], xlims = (4,4.03), ylims = (2.78,2.8), line=(:solid, :steppost), label="")

pc1 = plot(d1[:time], d1[Symbol("magLevNL.v")], xlims = (4,4.03), line=(:dot, :steppost), label="",xlabel="time [s]", ylabel="hold1.y [V]")
pc2 = plot!(d2[:time], d2[Symbol("magLevNL.v")], xlims = (4,4.03), line=(:dashdot, :steppost), label="")
pc3 = plot!(d3[:time], d3[Symbol("magLevNL.v")], xlims = (4,4.03),  line=(:solid, :steppost), label="")

pbc = plot(pb3,pc3,layout=(1,2))
p = plot(pa3,pbc, layout=(2,1))
#savefig(p,"Maglev_ADCPlot.pdf")

- Simulation reveals severe control degradation for the considered digital controller which is mainly due to ADC quantization effects of the Hall effect sensor output (measurement variable $e$).
- Simulation results for three ADC scenarios are plotted in the graph.
- The results indicate that using a suitable signal conditioning circuit mapping the relevant operating range ($[2.5\,\mathrm{V}, 3.5\,\mathrm{V}]$) to the full-scale voltage range of the Arduino ADC should significantly improve the control performance.

## 6. Real-Time Target Code

- First approach: Simply hand translate the Modelica code to Arduino C code.
- Second approach: Use OMC's experimental embedded code generation target together with AVR (Arduino) blocks from the Modelica_DeviceDrivers library.

<img src="images/MagLev_ACGModelDiagram.png" alt="The input model for the code generator consisting of the control algorithm and hardware related blocks." title="The input model for the code generator consisting of the control algorithm and hardware related blocks"/>
Input model for the code generator tuned at the actual demonstrator using a (non-ideal) signal conditioning circuit for measurement variable $e$.


Bernhard Thiele, Thomas Beutlich, Volker Waurich, Martin Sjölund, and Tobias Bellmann. Towards a Standard-Conform Platform-Generic and Feature-Rich Modelica Device Drivers Library. In In Jiří Kofránek and Francesco Casella, editors, _12th Int. Modelica Conference_, Prague, Czech Republic, May 2017. [doi:10.3384/ecp17132713](http://dx.doi.org/10.3384/ecp17132713).

## 7. Demonstrator

<img src="images/MagLev_Arduino.jpg" alt="TArduino controlled electromagnetic levitation system." title="Arduino controlled electromagnetic levitation system"/>

## Conclusions

- With a suitable Julia API for Modelica the two languages can complement each other.
- Simple web technology based GUIs for Modelica simulations can be created in a few lines of code.
- Further API enhancements, like adding a “find equilibrium” function, could reduce code duplication and increase user convenience.
- Cons: Julia's JIT compilation overhead when executing (plotting) code the "first time"
- The example can be readily reproduced, e.g., in the context of a lab session in control education